In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from transformers import RobertaTokenizer
import torch
from tqdm import tqdm
torch.cuda.empty_cache()
import numpy as np
from sklearn.metrics import f1_score

In [2]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

In [3]:
labels_dict = {'ANGRY':0, 
          'DISGUST':1, 
          'FEAR':2, 
          'HAPPY':3, 
          'SAD':4, 
          'SURPRISE':5}
df = pd.read_csv('../data/cleaned_emotion__.csv', encoding='ISO-8859-1')
df['label'] = df['emotion'].apply(lambda x: labels_dict[x])
print(len(df))
df.head()

1002


,text,emotion,label
0,THERE IS THE BITCH THAT SHOULD HAVE BEEN SHOT ...,DISGUST,1
1,"Tucker Carlson: âWe got to this sad, chaotic...",SAD,4
2,Its war time!,ANGRY,0
3,She is no doubt a fighter,ANGRY,0
4,SouthernBelle59 I donât care who they were! ...,SAD,4


In [4]:
text = list(df.text)
labels = list(df.label)
train_texts, val_texts, train_labels, val_labels = train_test_split(text, labels, test_size=.15, stratify=labels)

In [5]:
from torch.utils.data import TensorDataset
from transformers import BertTokenizer

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

In [7]:
train_encode = tokenizer.batch_encode_plus(
    train_texts,
    add_special_tokens=True,
    padding=True,
    truncation = True,
    max_length = 256,   
    return_tensors='pt'
)
valid_encode = tokenizer.batch_encode_plus(
    val_texts,
    add_special_tokens=True,
    padding=True,
    truncation = True,
    max_length = 256,   
    return_tensors='pt'
)

valid_input = valid_encode['input_ids']
valid_attention = valid_encode['attention_mask']
valid_labels = torch.tensor(val_labels)

input_ids = train_encode['input_ids']
attention_mask = train_encode['attention_mask']
labels = torch.tensor(train_labels)


In [8]:
train_data = TensorDataset(input_ids,
                          attention_mask,
                          labels)
valid_data = TensorDataset(valid_input,
                          valid_attention,
                          valid_labels)

In [9]:
from transformers import BertForSequenceClassification

#Finetuning pretrained BERT model with objective -> Sentence Classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels = 6,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [10]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 8

train_dataloader = DataLoader(train_data,
                              sampler = RandomSampler(train_data),
                              batch_size = BATCH_SIZE)
valid_dataloader = DataLoader(valid_data,
                              sampler = SequentialSampler(valid_data),
                              batch_size = BATCH_SIZE)

In [11]:
from transformers import AdamW, get_cosine_schedule_with_warmup

#Works better than torch.optim.Adam
optimizer = AdamW(model.parameters(),
                           lr=1e-5,
                           eps=1e-6,
                           weight_decay=1e-1)
#chosen after setting = 3 (f1 -> 0.71), 5 (f1 -> 0.77) but acpc for many classes is 0
#10 Epochs helps with generalization
EPOCHS = 10 

#Works better than torch.optim.CosineAnnealingLR and transformers.get_linear_schedule_with_warmup
scheduler = get_cosine_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps=len(train_dataloader)*EPOCHS)

In [12]:
def f1(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [13]:
#Per Class Accuracy
def acpc(preds, labels):
    code_dict = {val:key for key,val in labels_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for idx in np.unique(labels_flat):
        y_hat = preds_flat[labels_flat==idx]
        y = labels_flat[labels_flat==idx]
        print(f'Class: {code_dict[idx]}')
        print(f'Accuracy: {len(y_hat[y_hat==idx])/ len(y)}\n')

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [15]:
def evaluate(valid_dataloader):
    model.eval()

    total_eval_loss = 0
    y_hat, y = [], []

    for batch in valid_dataloader:
        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids' : batch[0],
                'attention_mask': batch[1],
                'labels' : batch[2]
                }
        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        y_hat.append(logits)
        y.append(label_ids)

    avg_eval_loss = total_eval_loss/len(valid_dataloader) 

    y_hat = np.concatenate(y_hat, axis=0)
    y = np.concatenate(y, axis=0)
            
    return avg_eval_loss, y_hat, y

In [16]:
vloss = []
score = []
for epoch in tqdm(range(1, EPOCHS+1)):
    model.train()

    total_loss = 0

    progress_bar = tqdm(train_dataloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids' : batch[0],
                  'attention_mask': batch[1],
                  'labels' : batch[2]
                  }
        outputs = model(**inputs)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    torch.save(model.state_dict(), f'./bert emotion/BERT_emotion_epoch_{epoch}.pt')

    tqdm.write(f'\nEpoch {epoch}')

    avg_training_loss = total_loss/len(train_dataloader)            
    tqdm.write(f'Training loss: {avg_training_loss}')

    val_loss, predictions, actual = evaluate(valid_dataloader)
    score_f1 = f1(predictions, actual)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {score_f1}')
    score.append(score_f1)
    vloss.append(val_loss)

Epoch 1:  30%|███████████████▎                                   | 32/107 [00:17<00:41,  1.82it/s, training_loss=0.587]


Epoch 1:  61%|██████████████████████████████▉                    | 65/107 [00:35<00:23,  1.82it/s, training_loss=0.551]


Epoch 1:  92%|██████████████████████████████████████████████▋    | 98/107 [00:54<00:05,  1.76it/s, training_loss=0.485]


  0%|                                                                                           | 0/10 [01:00<?, ?it/s]


Epoch 1
Training loss: 1.7274348178756571


...

Epoch 10:  93%|██████████████████████████████████████████████▎   | 99/107 [00:54<00:04,  1.82it/s, training_loss=0.026]


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [10:19<01:02, 62.02s/it]


Epoch 10
Training loss: 0.08788882161133757


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [10:22<00:00, 62.21s/it]

Validation loss: 0.2338875912522015
F1 Score (Weighted): 0.9275187446348893


In [17]:
for i, item in enumerate(zip(score, vloss)):
    print(i, item[0], item[1])

0 0.45836097204321613 1.604380871120252
1 0.6290257648783059 1.234389198453803
2 0.8813091459250199 0.6669007916199533
3 0.9140456783834268 0.38307346795734604
4 0.9276403164424202 0.29115622294576543
5 0.9276403164424202 0.2646070569753647
6 0.9208962280785978 0.24420037865638733
7 0.9275187446348893 0.2359455665083308
8 0.9341412611911806 0.23399121431927933
9 0.9275187446348893 0.2338875912522015


In [19]:
model.to(device)
model.load_state_dict(torch.load('./bert emotion/BERT_emotion_epoch_9.pt', map_location=torch.device('cpu')))

_, predictions, actual = evaluate(valid_dataloader)
print(acpc(predictions, actual))
labs = {v:k for k,v in labels_dict.items()}
for i in range (len(predictions)):
  print(labs[np.argmax(predictions[i])], val_texts[i])
  print()
    


Class: ANGRY
Accuracy: 0.96

Class: DISGUST
Accuracy: 0.92

Class: FEAR
Accuracy: 0.92

Class: HAPPY
Accuracy: 0.96

Class: SAD
Accuracy: 0.9230769230769231

Class: SURPRISE
Accuracy: 0.92

None
SURPRISE Just curious did anyone else get a $600 stimulus payment in the last week or so??

FEAR The deep state is afraid of Trumps support. They will try to take him out.

FEAR This is terrifying!!

HAPPY Thank you!

SURPRISE He's got history credentials?

HAPPY Happy to be #american #godblessamerica

ANGRY "They're not going to take this White House, we're gonna fight like Hell." -President Trump (1/4/2021)

DISGUST Disgusting piece of garbage.

DISGUST What scum

HAPPY Triste verdade.

HAPPY â¤ï¸ love this.

ANGRY This was Antifa! No doubt.

FEAR Tomorrow is a big day. My support and prayers go out to all exercising their right to assemble in DC today and especially tomorrow. This is a chance to show everyone that we won't back down. No doubt, we will see the media calling this a "far righ